In [1]:
#import necessary libraries
import json
import pandas as pd
import numpy as np

In [2]:
#Reading from location on cluster, change the file_location accordingly
#df = pd.read_json("https://s3-eu-west-1.amazonaws.com/dwh-test-resources/recipes.json",lines=True)
#File location and type
file_location = "/FileStore/tables/recipes.json"
file_type = "json"

# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)
df.dtypes


cookTime,datePublished,description,image,ingredients,name,prepTime,recipeYield,url
PT,2013-04-01,Got leftover Easter eggs? Got leftover Easter ham? Got a hearty appetite? Good! You've come to the right place! I...,http://static.thepioneerwoman.com/cooking/files/2013/03/leftoversandwich.jpg,"12 whole Hard Boiled Eggs 1/2 cup Mayonnaise 3 Tablespoons Grainy Dijon Mustard Salt And Pepper, to taste Several Dashes Worcestershire Sauce Leftover Baked Ham, Sliced Kaiser Rolls Or Other Bread Extra Mayonnaise And Dijon, For Spreading Swiss Cheese Or Other Cheese Slices Thinly Sliced Red Onion Avocado Slices Sliced Tomatoes Lettuce, Spinach, Or Arugula",Easter Leftover Sandwich,PT15M,8,http://thepioneerwoman.com/cooking/2013/04/easter-leftover-sandwich/
PT10M,2011-06-06,I finally have basil in my garden. Basil I can use. This is a huge development. I had no basil during the winter. None. G...,http://static.thepioneerwoman.com/cooking/files/2011/06/pesto.jpg,"3/4 cups Fresh Basil Leaves 1/2 cup Grated Parmesan Cheese 3 Tablespoons Pine Nuts 2 cloves Garlic, Peeled Salt And Pepper, to taste 1/3 cup Extra Virgin Olive Oil 1/2 cup Heavy Cream 2 Tablespoons Butter 1/4 cup Grated Parmesan (additional) 12 ounces, weight Pasta (cavitappi, Fusili, Etc.) 2 whole Tomatoes, Diced",Pasta with Pesto Cream Sauce,PT6M,8,http://thepioneerwoman.com/cooking/2011/06/pasta-with-pesto-cream-sauce/
PT15M,2011-09-15,"This was yummy. And easy. And pretty! And it took basically no time to make. Before I share the recipe, I'll just say it:...",http://static.thepioneerwoman.com/cooking/files/2011/09/porkloin.jpg,"2 whole Pork Tenderloins Salt And Pepper, to taste 8 Tablespoons Herbs De Provence (more If Needed 1 cup Preserves (fig, Peach, Plum) 1 cup Water 1 Tablespoon Vinegar",Herb Roasted Pork Tenderloin with Preserves,PT5M,12,http://thepioneerwoman.com/cooking/2011/09/herb-roasted-pork-tenderloin-with-preserves/
PT20M,2012-04-23,"I made this for a late lunch Saturday, and it absolutely completed me. It also absolutely completed my fourteen-year-old daug...",http://static.thepioneerwoman.com/cooking/files/2012/04/florentine.jpg,"1 pound Penne 4 whole Boneless, Skinless Chicken Breasts Salt And Pepper, to taste 2 Tablespoons Butter 2 Tablespoons Olive Oil 4 cloves Garlic, Minced 3/4 cups Dry White Wine 3/4 cups Low-sodium Broth, More If Needed 1 bag Baby Spinach 2 cups Grape Tomatoes, Halved Lengthwise 4 ounces, weight Parmesan Cheese, Shaved With Vegetable Peeler",Chicken Florentine Pasta,PT10M,10,http://thepioneerwoman.com/cooking/2012/04/chicken-florentine-pasta/
PT,2011-06-13,"Iced coffee is my life. When I wake up, often around the time party animals on the west coast are just heading home, I start ...",http://static.thepioneerwoman.com/cooking/files/2011/06/icedcoffee.jpg,"1 pound Ground Coffee (good, Rich Roast) 8 quarts Cold Water Half-and-half (healthy Splash Per Serving) Sweetened Condensed Milk (2-3 Tablespoons Per Serving) Note: Can Use Skim Milk, 2% Milk, Whole Milk, Sugar, Artificial Sweeteners, Syrups...adapt To Your Liking!",Perfect Iced Coffee,PT8H,24,http://thepioneerwoman.com/cooking/2011/06/perfect-iced-coffee/
PT10M,2012-05-31,"When I was in Albuquerque with Marlboro Man and the boys a month ago, I had a really fun book signing. Such incredibly nice a...",http://static.thepioneerwoman.com/cooking/files/2012/05/enchilada.jpg,"1 whole Onion, Diced 2 Tablespoons Butter 1 can (15 Ounce) Green Enchilada Sauce 2 cans (4 Ounce) Chopped Green Chilies 12 whole Corn Tortillas 2 cups Freshly Grated Cheddar (or Cheddar-jack) Cheese (or Any Cheese You'd Like) Sour Cream Salsa Pico De Gallo (optional) Guacamole (optional) Cilantro Leaves, Optional",Easy Green Chile Enchiladas,PT5M,4,http://thepioneerwoman.com/cooking/2012/05/easy-green-chile-enchiladas/
PT5M,2013-03-25,"Imagine the Easter Bunny laying an egg. Wait. That’s not anatomically possible. And anyway, the Easter Bunny is a b...",http://static.thepioneerwoman.com/cooking/files/2013/03/DSC_9637.jpg,

In [3]:
#Function to extract recipes that contain beef.
#Optionally can done fully using lambda
def findingbeef(x:str):
    if 'beef' in x:
        return x
    elif 'Beef' in x:
        return x
    else:
        return 'not beef'

In [4]:
#working will continue on newdf created (note** using spark thus lazy evaluation implicitly happening unless invoking action)
#Option 1
#df.createOrReplaceTempView("newrecipes")
#newdf = spark.sql("select * from newrecipes where lcase(ingredients) like '%beef%'")
#newdf.count()

#Option 2 using udf
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
udf = UserDefinedFunction(lambda x: findingbeef(x), StringType())
newdf = df.withColumn('selrecipe',udf(df.ingredients))
newdf = newdf.filter(newdf['selrecipe'] != 'not beef')
newdf.count()
#newdf.columns
#newdf.select("selreci").show(1,truncate=False)



Out[304]: 47

In [5]:
#newdf.groupBy("cookTime").count().show()
#newdf.groupBy("prepTime").count().show()
#or
from pyspark.sql.functions import countDistinct
newdf.select(countDistinct("cookTime")).show()
newdf.select(countDistinct("prepTime")).show()

#To look at unique values for these fields
#newdf.select("cookTime").distinct().collect()
#newdf.select("prepTime").distinct().collect()

+------------------------+
count(DISTINCT cookTime)|
+------------------------+
 19|
+------------------------+

+------------------------+
count(DISTINCT prepTime)|
+------------------------+
 11|
+------------------------+

In [6]:
#Using udf to strip out unnecessary characters(for easier calculation later)
#optionally instead of newcolumn names, same column names can be used..(better approach when data is huge)

udf = UserDefinedFunction(lambda x: x[2:], StringType())
newdf = newdf.withColumn('newprepTime',udf(newdf.prepTime))
newdf = newdf.withColumn('newcookTime',udf(newdf.cookTime))
newdf.select("newcookTime").show(2)
newdf.select("newprepTime").show(2)


+-----------+
newcookTime|
+-----------+
 25M|
 3H|
+-----------+
only showing top 2 rows

+-----------+
newprepTime|
+-----------+
 10M|
 20M|
+-----------+
only showing top 2 rows

In [7]:
#for conversion of fields into computable time duration in minutes
#using timedelta
def timeff(x):
    n = pd.to_timedelta(x)
    return n/np.timedelta64(1, 'm')


In [8]:
#before applying function,confirming datatypes
#newdf.dtypes

udfx = UserDefinedFunction(lambda x: timeff(x), StringType())
newdf = newdf.withColumn('newprepTime1',udfx(newdf.newprepTime))
newdf = newdf.withColumn('newcookTime1',udfx(newdf.newcookTime))

In [9]:
newdf.select("prepTime","newPrepTime","newPrepTime1").show(2)

+--------+-----------+------------+
prepTime|newPrepTime|newPrepTime1|
+--------+-----------+------------+
 PT10M| 10M| 10.0|
 PT20M| 20M| 20.0|
+--------+-----------+------------+
only showing top 2 rows

In [10]:
newdf.select("prepTime","newPrepTime","newPrepTime1").dtypes

Out[310]: [('prepTime', 'string'), ('newPrepTime', 'string'), ('newPrepTime1', 'string')]

In [11]:
#assessing difficulty level based on time spent
def difficulty(x):
    if x<30:
        return 'easy'
    elif x>=30 and x<=60:
        return 'medium'
    else:
        return 'hard'

In [12]:
#applying function
udfn = UserDefinedFunction(lambda x: difficulty(x), StringType())
from pyspark.sql.types import FloatType
newdf = newdf.withColumn("newcookTime1",newdf["newcookTime1"].cast(FloatType()))
newdf = newdf.withColumn("newprepTime1",newdf["newprepTime1"].cast(FloatType()))
newdf = newdf.withColumn('TotalTime',(newdf.newcookTime1+newdf.newprepTime1))
newdf = newdf.withColumn('difficulty',udfn(newdf.TotalTime))
newdf.select("newprepTime1","newcookTime1","difficulty").show(5)
                                                            

+------------+------------+----------+
newprepTime1|newcookTime1|difficulty|
+------------+------------+----------+
 10.0| 25.0| medium|
 20.0| 180.0| hard|
 5.0| 25.0| medium|
 10.0| 20.0| medium|
 5.0| 240.0| hard|
+------------+------------+----------+
only showing top 5 rows

In [13]:
#looking at results and saving it as csv in one file..
Result = newdf.groupby(['difficulty']).agg({"newcookTime1": "avg"}).withColumnRenamed("avg(newcookTime1)","avg_total_cooking_time" )
Result.show()
Result.coalesce(1).write.format("csv").save("Finalreport.csv")



+----------+----------------------+
difficulty|avg_total_cooking_time|
+----------+----------------------+
 medium| 28.125|
 hard| 169.7826086956522|
 easy| 9.625|
+----------+----------------------+